In [14]:
import numpy as np
import pandas as pd

In [8]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['let your neural network be polyglot',
     'use multilingual embeddings!'],
    lang='en')  # lang is only used for tokenization

# embeddings is a N*1024 (N = number of sentences) NumPy array

In [9]:
embeddings.shape

(2, 1024)

In [10]:
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.'],
    lang=['en', 'fr', 'de'])

In [40]:
def get_correlation(src_sentence,tgt_sentence):
    embeddings = laser.embed_sentences([src_sentence, tgt_sentence], lang=['en', 'de'])
    src_emb, tgt_emb = embeddings[0], embeddings[1]
    corr = (src_emb / np.linalg.norm(src_emb)).dot(tgt_emb / np.linalg.norm(tgt_emb))
    return corr

In [16]:
dataset = pd.read_pickle('../data/dataset_correlations_v2.pickle')

In [41]:
dataset['sentence_correlation'] = dataset.apply(lambda row: 
                                  get_correlation(row["sentences_en"], row["sentences_ge"]),axis=1)

/Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/ATen/native/RangeFactories.cpp:153: UserWarning: The number of elements in the out tensor of shape [50] is 50 which does not match the computed number of elements 52. Note that this may occur as a result of rounding error. The out tensor will be resized to a tensor of shape (52,).
/Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/ATen/native/RangeFactories.cpp:153: UserWarning: The number of elements in the out tensor of shape [52] is 52 which does not match the computed number of elements 61. Note that this may occur as a result of rounding error. The out tensor will be resized to a tensor of shape (61,).


In [43]:
dataset['sentence_correlation'].corr(dataset['scores'])

0.0568468439418905

In [47]:
# dataset.to_pickle('../data/dataset_corrleations_laser.pickle')